In [1]:

import pandas as pd
import numpy as np

import shutil
import sys
import os

# yoltv4_path = '/home/carlos/Python_Workspace/pcb-cv_detection/'
# sys.path.append(os.path.join(yoltv4_path, 'yoltv4'))
# #import yoltv4.prep_train
# #import yoltv4.tile_ims_labels as tile_ims_labels
# #import yoltv4.post_process
# #reload(prep_train)

from yolov3.utils import detect_image, detect_realtime, detect_video, Load_Yolo_model, detect_video_realtime_mp
from yolov3.configs import *

from yolt_extension import rescale, slice_images, post_process

In [2]:
## TEST Tensorflow Configuration

import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')

print(gpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [7]:
### CONFIGURATION ###

# set dataset path
#data_root = '/home/carlos/Python_Workspace/DATA/Component_Detection/TEST_DATA'
#data_root = "C:\\Users\\carlo\\Python_workspace\\DATA\\TEST_DATA"
data_root = "../DATA/TEST_DATA"


# Path of classes name file
#classname_file_path =  '/home/carlos/Python_Workspace/DATA/Component_Detection/components.name'

# Image extension
img_ext = ".jpg"

grid_pixels_density = [12,10,7]
min_size_density = [0, 2,32  ]
max_size_density = [2, 32,1000]

grid_pixels_density = [15,9,6]
min_size_density = [0,2.5,35]
max_size_density = [2.5,35,10000]

In [8]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

#Function for obtaining the images in a folder
def get_images_dicts(directory):
    dataset_dicts = []
    for filename in [file for file in listdir_nohidden(directory) if not(file.endswith(".csv"))]:      
        dataset_dicts.append(filename)

    return dataset_dicts

-----
## Prepare Data

Divide the scaled images in slices of 416x416 with stamps for recombining the data after.

In [8]:
# SLICE CONFIGURATION #
sliceHeight, sliceWidth = 416, 416
slice_overlap=0.2
out_ext = img_ext
verbose = True

# Folders for saving the temp data
data_root_tmp = data_root
outdir_root = os.path.join(data_root_tmp, 'process_tmp_folder')
outdir_ims = os.path.join(outdir_root, 'images_slice')
outdir_txt = os.path.join(outdir_root, 'txt')

# If the folder exists it will be deleted
if os.path.exists(outdir_ims) != 0:
    print("Removing existing data")
    shutil.rmtree(outdir_ims)

slice_images(im_scaled_dir,outdir_ims, sliceHeight, sliceWidth,slice_overlap,out_ext,verbose)

#################

# make list of all generated images path
im_list_test = []
for f in sorted([z for z in os.listdir(outdir_ims) if z.endswith(out_ext)]):
    im_list_test.append(f)

# Create the save folder if it does not exist
if os.path.exists(outdir_txt) == 0:
    os.makedirs(outdir_txt)

# Save the list with the path of all images
df_imgs_list = pd.DataFrame({'image': im_list_test})
df_imgs_list.to_csv(os.path.join(outdir_txt, 'imgs_list.txt'), header=False, index=False)



0 / 12 Beagleboard-C4_15.jpg h, w = 1176 1143
image.shape: (1176, 1143, 3)
Num slices: 16 sliceHeight 416 sliceWidth 416
Time to slice C:\Users\carlo\Python_workspace\DATA\TEST_DATA\images_scaled\Beagleboard-C4_15.jpg 0.13812541961669922 seconds
1 / 12 Beagleboard-C4_6.jpg h, w = 470 457
image.shape: (470, 457, 3)
Num slices: 4 sliceHeight 416 sliceWidth 416
Time to slice C:\Users\carlo\Python_workspace\DATA\TEST_DATA\images_scaled\Beagleboard-C4_6.jpg 0.034032583236694336 seconds
2 / 12 Beagleboard-C4_9.jpg h, w = 705 685
image.shape: (705, 685, 3)
Num slices: 9 sliceHeight 416 sliceWidth 416
Time to slice C:\Users\carlo\Python_workspace\DATA\TEST_DATA\images_scaled\Beagleboard-C4_9.jpg 0.07507014274597168 seconds
3 / 12 Beagleboard_Zippy_15.jpg h, w = 1143 1143
image.shape: (1143, 1143, 3)
Num slices: 16 sliceHeight 416 sliceWidth 416
Time to slice C:\Users\carlo\Python_workspace\DATA\TEST_DATA\images_scaled\Beagleboard_Zippy_15.jpg 0.14713311195373535 seconds
4 / 12 Beagleboard_Zipp

___
## Execution on Yolo

In this section we run Yolo over each of the images previously sliced. This part is based in the Yolo implemetation over TensorFlow from: [TensorFlow-2.x-YOLOv3](https://github.com/pythonlessons/TensorFlow-2.x-YOLOv3)

In [9]:

outdir_localPred = os.path.join(outdir_root, 'local_predictions')

# If the folder contains data it will be deleted
if os.path.exists(outdir_localPred) != 0:
    print("Removing existing data")
    shutil.rmtree(outdir_localPred)

# Create the save folder if it does not exist
if os.path.exists(outdir_localPred) == 0:
    os.makedirs(outdir_localPred)


arr_pred = np.empty((7, 0))

#Load Yolo Model on memory
yolo = Load_Yolo_model()

print("Model Loaded")

for image_name in df_imgs_list['image']:
    image_path = os.path.join(outdir_ims, image_name)
    save_path = os.path.join(outdir_localPred, image_name)

    #Image name without extension
    img_name = image_name.split(".")[0]

    lst_elements_detected = detect_image(yolo, image_path, save_path, input_size=YOLO_INPUT_SIZE, show=False, CLASSES=YOLO_COCO_CLASSES, score_threshold=0.1, rectangle_colors=(255,0,0))

    if(len(lst_elements_detected)>0):

        elements_detected = np.array(lst_elements_detected)

        array_image_name = np.tile(np.array([image_name]),(elements_detected.shape[0]))

        # print(array_image_name.shape)
        # print(elements_detected.shape)
        # print(elements_detected)

        new_pred = np.concatenate(  [[array_image_name], 
                                    [elements_detected[:,4]], 
                                    [elements_detected[:,5]],
                                    [elements_detected[:,0]],
                                    [elements_detected[:,1]],
                                    [elements_detected[:,2]],
                                    [elements_detected[:,3]]],
                                    axis = 0)
        
        #print(arr_pred.shape)
        #print(new_pred.shape)

        arr_pred = np.append(arr_pred, new_pred, axis=1)

        

    print("Processed image: "+image_name)
    


Loading Darknet_weights from: model_data/yolov3-PCBCV.weights
Model Loaded
Processed image: Beagleboard-C4_15__0_0_416_416_0_1143_1176.jpg
Processed image: Beagleboard-C4_15__0_332_416_416_0_1143_1176.jpg
Processed image: Beagleboard-C4_15__0_664_416_416_0_1143_1176.jpg
Processed image: Beagleboard-C4_15__0_727_416_416_0_1143_1176.jpg
Processed image: Beagleboard-C4_15__332_0_416_416_0_1143_1176.jpg
Processed image: Beagleboard-C4_15__332_332_416_416_0_1143_1176.jpg
Processed image: Beagleboard-C4_15__332_664_416_416_0_1143_1176.jpg
Processed image: Beagleboard-C4_15__332_727_416_416_0_1143_1176.jpg
Processed image: Beagleboard-C4_15__664_0_416_416_0_1143_1176.jpg
Processed image: Beagleboard-C4_15__664_332_416_416_0_1143_1176.jpg
Processed image: Beagleboard-C4_15__664_664_416_416_0_1143_1176.jpg
Processed image: Beagleboard-C4_15__664_727_416_416_0_1143_1176.jpg
Processed image: Beagleboard-C4_15__760_0_416_416_0_1143_1176.jpg
Processed image: Beagleboard-C4_15__760_332_416_416_0_114

In [10]:
print(arr_pred.shape)

print(arr_pred[:,0:4])

(7, 900)
[['Beagleboard-C4_15__0_0_416_416_0_1143_1176.jpg'
  'Beagleboard-C4_15__0_0_416_416_0_1143_1176.jpg'
  'Beagleboard-C4_15__0_0_416_416_0_1143_1176.jpg'
  'Beagleboard-C4_15__0_0_416_416_0_1143_1176.jpg']
 ['0.9118691682815552' '0.7795941829681396' '0.21386736631393433'
  '0.10525654256343842']
 ['0.0' '0.0' '0.0' '0.0']
 ['209.7405242919922' '323.1295166015625' '304.01666259765625'
  '206.7677764892578']
 ['209.08082580566406' '248.5139923095703' '0.0' '53.367530822753906']
 ['229.63673400878906' '376.59765625' '415.0' '266.0434875488281']
 ['217.3227996826172' '297.1427917480469' '118.43515014648438'
  '160.68572998046875']]


In [11]:
df_predictions = pd.DataFrame(np.rot90(arr_pred),columns=['im_name','prob','category','Xmin', 'Ymin', 'Xmax', 'Ymax'])
df_predictions = df_predictions.astype({'im_name':str,'prob':float,'category':str,'Xmin':float, 'Ymin':float, 'Xmax':float, 'Ymax':float})
df_predictions.head(10)

,im_name,prob,category,Xmin,Ymin,Xmax,Ymax
0,IMAC_9__996_996_416_416_0_3083_1886.jpg,0.214000,0.0,97.131821,19.569674,104.707825,32.775295
1,IMAC_9__996_996_416_416_0_3083_1886.jpg,0.273736,0.0,203.548431,65.510124,221.759521,75.965080
2,IMAC_9__996_996_416_416_0_3083_1886.jpg,0.294081,0.0,204.227173,47.964779,222.651306,58.167057
3,IMAC_9__996_996_416_416_0_3083_1886.jpg,0.355342,0.0,127.876907,315.986877,184.751694,366.969543
4,IMAC_9__996_996_416_416_0_3083_1886.jpg,0.418865,0.0,31.752892,99.167763,52.998810,110.445717
5,IMAC_9__996_996_416_416_0_3083_1886.jpg,0.449766,0.0,10.466293,33.006310,76.045486,84.931297
6,IMAC_9__996_996_416_416_0_3083_1886.jpg,0.465630,0.0,59.185635,330.366760,92.555023,392.373840
7,IMAC_9__996_996_416_416_0_3083_1886.jpg,0.820481,0.0,324.473145,313.115662,383.032349,367.964417
8,IMAC_9__996_664_416_416_0_3083_1886.jpg,0.459643,0.0,121.362617,345.097626,235.129822,390.049591
9,IMAC_9__996_664_416_416_0_3083_1886.jpg,0.483025,0.0,341.802734,32.167709,408.546509,83.787964


-----
## Process the results

In [12]:
from detector import  post_process_predictions

###### CONFIGURATION #########
raw_im_dir=im_scaled_dir
im_ext='.jpg'
out_dir_root=os.path.join(outdir_root, 'yolt_predictions')
detection_thresh=0.2
slice_size=416
n_plots = 20
max_edge_aspect_ratio=2.5
nms_overlap_thresh=0.5

df_yolt_predictions = post_process_predictions(df_predictions,
                        raw_im_dir,
                        im_ext,
                        out_dir_root,
                        detection_thresh,
                        max_edge_aspect_ratio,
                        nms_overlap_thresh,
                        slice_size,
                        n_plots
                        )

Augmenting dataframe of initial length: 725 ...
0


c:\Users\carlo\Python_workspace\PCB-CV\Component Detection\detector.py:418: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predictions['image_path'] = im_path_list
c:\Users\carlo\Python_workspace\PCB-CV\Component Detection\yoltv4\post_process.py:709: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Image_Root'] = im_roots
c:\Users\carlo\Python_workspace\PCB-CV\Component Detection\yoltv4\post_process.py:710: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

removing bad idxs near junctions: [8, 18, 30, 44, 62, 66, 71, 75, 82, 88, 103, 107, 110, 115, 126, 129, 134, 136, 142, 148, 150, 158, 167, 190, 201, 211, 212, 229, 236, 243, 248, 258, 259, 262, 298, 318, 322, 328, 333, 348, 349, 359, 372, 388, 390, 393, 395, 397, 398, 400, 402, 413, 418, 420, 433, 438, 440, 473, 477, 490, 491, 501, 503, 506, 515, 516, 520, 533, 547, 552, 554, 568, 570, 573, 575, 578, 598, 602, 606, 607, 608, 618, 619, 620, 621, 622, 623, 629, 630, 632, 635, 636, 638, 641, 646, 651, 652, 653, 658, 663, 666, 668, 669, 672, 675, 679, 681, 687, 690, 691, 692, 698, 699, 700, 705, 706, 707, 710, 711, 712, 713, 722, 723, 725, 729, 730, 731, 732, 734, 736, 744, 746, 762, 768, 769, 770, 774, 779, 780, 781, 786, 802, 809, 881, 892, 895, 897]
Time to augment dataframe of length: 578 = 0.09508585929870605 seconds
Running refine_df()...
Executing non-max suppression...
  non-max suppression init boxes: 33
  non-max suppression final boxes: 21
Executing non-max suppression...
  non-

-----
## Combine results with differents scales

In [13]:
df_columns = ["Center_X","Center_Y","Width","Height","Prob","Level_1","Prob_L1","Level_2","Prob_L2","Level_3","Prob_L3","Class","Prob_Clas"]

images = get_images_dicts(im_dir)


#Loop for processing each image of the PCB found in the images folder
for img_filename in images:
    
    #Create the predictions DF for the board
    RES_DF = pd.DataFrame(columns=df_columns)

    #Image name without extension
    img_name = img_filename.split(".")[0]

    for index, pixel_density in enumerate(grid_pixels_density):

        img_scaled_name = img_name + "_" + str(pixel_density)
        print(img_scaled_name)

        # Exclude the predictions different from the current iteration data
        current_df = df_yolt_predictions.loc[df_yolt_predictions["im_name_root"]== (img_scaled_name)]

        DF = pd.DataFrame(columns=df_columns)

        # Calculate the size of the box in pixels
        DF["Width"] = current_df["Xmax_Glob"] - current_df["Xmin_Glob"]
        DF["Height"] = current_df["Ymax_Glob"] -current_df["Ymin_Glob"]
        DF["Center_X"] = current_df["Xmin_Glob"] + (DF["Width"] /2)
        DF["Center_Y"] = current_df["Im_Height"] - (current_df["Ymin_Glob"] + (DF["Height"] /2))

        # Calculate size in mm
        DF["Width"] = DF["Width"] / pixel_density
        DF["Height"] = DF["Height"] / pixel_density
        DF["Area"] = DF["Width"] * DF["Height"]
        DF["Center_X"] = DF["Center_X"] / pixel_density
        DF["Center_Y"] = DF["Center_Y"] / pixel_density

        DF["Prob"] = current_df["prob"] 

        pcbName = current_df["im_name_root"].iloc[0]
        print(pcbName)

        # Round the dataframe to 2 decimals
        DF = DF.round(3)

        # Exclude the detections outside the detection range
        DF = DF[(DF["Area"]<=max_size_density[index])] 
        DF = DF[(DF["Area"]>min_size_density[index])]

        DF = DF.drop(columns=["Area"])

        # Append the prediciton in the board DF
        RES_DF =  RES_DF.append(DF,ignore_index=True)

    # Save dataset
    df_path = os.path.join(data_root, img_name + ".csv")
    RES_DF.to_csv(df_path,index=True)




Beagleboard-C4_15
Beagleboard-C4_15
Beagleboard-C4_9
Beagleboard-C4_9
Beagleboard-C4_6
Beagleboard-C4_6
Beagleboard_Zippy_15
Beagleboard_Zippy_15
Beagleboard_Zippy_9
Beagleboard_Zippy_9
Beagleboard_Zippy_6
Beagleboard_Zippy_6
GTC-MAIN_15
GTC-MAIN_15
GTC-MAIN_9
GTC-MAIN_9
GTC-MAIN_6
GTC-MAIN_6
IMAC_15
IMAC_15
IMAC_9
IMAC_9
IMAC_6
IMAC_6
